# Classes: Class Objects vs. Instance Objects

In the previous tutorial, we have learned how to define a custom class and create multiple instances from this class.
In this tutorial, we will look at the difference between so-called class objects and instance objects.

Let's take a second at the `Person` class from the previous tutorial.

In [178]:
class Person:

    def __init__(self, first_name, last_name):
        self.first_name = first_name
        self.last_name = last_name
        
        
    def full_name(self):
        return f'{self.first_name} {self.last_name}'
    
    def __str__(self):
        return f'{self.first_name} {self.last_name}' 
    
    def __repr__(self):
        return f'Person(first_name="{self.first_name}", last_name="{self.last_name}")' 

As can be seen all method definitions have `self` as the first argument, and we use this variable in the method body to access different attributes. When we call one of these methods, the object itself is automatically passed in as first argument. This access the object's properties from inside the object's methods.

In some languages this parameter is implicit (it is not visible in the function signature ). In Python it is explicitly exposed by the `self` parameter..

**What happens internally when the Python interpreter encounters a new class definition (such as the one below)? How many objects related to the class `Person` exist once we executed the cell?**

In [90]:
class Person:
    
    pass

You might be tempted say that no objects were created at this point since no object were instantiated from class `Person` (we call it's default constructor). <br/>
However, this is not true. In fact, when the Python Interpreter finds the class definition, it automatically created as an object for this class --- a **class object**.
                                                                                                                    

We can access the class object simply by it's class name. For example, we can print the class object.

In [91]:
# A new class object called "Person" was created in the previous cell.
# We access the class by the class name.
print(Person)

<class '__main__.Person'>


**Class objects support two kinds of operations: attribute references and instantiation**

Well, that's an interesting observation. Next, let's take a closer look on what happens when we create a new instance of class `Person`. <br/>
In other words, what happens if we run the following line of code ...

In [92]:
p = Person()

`Person` represents the class object. By definition, when the `()` operator is called on a class object, first a new empty object (referred to as an instance object) is created (*) and passed to the init method `__init__()` of the class object. The init method receives the instance object as the first argument which allowes setting attributes of the instance object. Finally, the instance object is returned by the instance method and assigned to the local variable (which in our case is called `p`).

(*) Internally, the `__new__()` method is called which creates a new empty object.

## Class Attributes

Since you now know that there is a difference between instance objects and class objects, you might wonder whether class objects can have attributes. Yes, this is indeed possible.

In the following, the class `Person` (and so the `Person` class object) is extended with an additional class attribute called `TITLES`.

In [152]:
class Person:
    
    TITLES = ('Mr', 'Ms', 'Mrs')

    def __init__(self, first_name, last_name):
        self.first_name = first_name
        self.last_name = last_name
        
        
    def full_name(self):
        return f'{self.first_name} {self.last_name}'
    
    def __str__(self):
        return f'{self.first_name} {self.last_name}' 
    
    def __repr__(self):
        return f'Person(first_name="{self.first_name}", last_name="{self.last_name}")' 
    

Class attributes are often used to define constants which are closely associated with a particular class. Class attributes can either be accessed via a class instance, or via the class directly which does not require to create a new instance.

In [153]:
p = Person('Mike', 'Tyson')

In [154]:
# We can access a class attribute via an instance
print(p.TITLES)

# but we can also access via the object class
print(Person.TITLES)

('Mr', 'Ms', 'Mrs')
('Mr', 'Ms', 'Mrs')


Next, let's continue with a small experiment. Let' create two instances of the class `Person` but alter the attribute of the class object in between.

In [157]:
print('Class object', Person.TITLES)
p1 = Person('Person', '1')
print(p1, p1.TITLES)
print('=================')

# Let's alter the class attribute of the class object Person
p.TITLES = ['Mr', 'Ms', 'Mrs', '-']
p2 = Person('Person', '2')

print('Class object', Person.TITLES)
print(p1, p1.TITLES)
print(p2, p2.TITLES)

Class object ('Mr', 'Ms', 'Mrs')
Person 1 ('Mr', 'Ms', 'Mrs')
Class object ('Mr', 'Ms', 'Mrs')
Person 1 ('Mr', 'Ms', 'Mrs')
Person 2 ('Mr', 'Ms', 'Mrs')


Oh interesting, appearently when we change `TITLES` attribute of the class object at runtime, we also change the `TITLES` attribute of ALL instances.
<br/>
So, it seems that class attributes are shared among all instances of a class.

Consequently, if class attributes are shared, we should also be able to modify the class attribute via a specific instance. 

Well, let's try ...

In [163]:
# Reset class attribute
p.TITLES = ('Mr', 'Ms', 'Mrs')

# Create a new person
p = Person('Mike', 'Tyson')

# Access the class attribute via the instance p and the object class
# Both should be ('Mr', 'Ms', 'Mrs') at this point
print('Class object', Person.TITLES)
print(p, p.TITLES)
print('=================')

# Let's try to change the class attribute via the instance p
p.TITLES = ('Mr', 'Ms', 'Mrs', '-')

# The value of the class attribute should now be ()'Mr', 'Ms', 'Mrs', '-')
print(p, p.TITLES)

# The value of Person.TITLES ... Oh, it's still ('Mr', 'Ms', 'Mrs')
print('Class object:', Person.TITLES)

Class object ('Mr', 'Ms', 'Mrs')
Mike Tyson ('Mr', 'Ms', 'Mrs')
Mike Tyson ('Mr', 'Ms', 'Mrs', '-')
Class object: ('Mr', 'Ms', 'Mrs')


Ok, that was unexpected. Apparently, modifying `TITLES` via an instance does not alter the class object. Well, but then this indicates that class attributes are NOT shared, doesn't it?

### The ICPO Lookup

The answer to the question what's going on here is provided by what's called ICPO (Instance - Class - Parent - Object) lookup by Reuven M. Lerner (Python book author). In fact, class attributes are NOT shared (*) but the ICPO can easily result in the impression that they are. Python executes the following search in order to retrieve the value of an attribute:

1. **Instance**: Instance level check. We ask whether the instance *p* has an attribute `TITLES`. If so, then `TITLES` is retrieved from the instance *p* and the search ends.
2. **Class**: If Python doesn’t find `TITLES` on the instance, then it looks for `type(p).TITLES` which is the class object. If it finds the attribute here, then it returns the value, and the search ends.
3. **Parents**: If Python would not have found `TITLES` on `type(p)`, then it had look on the parents of `type(p)`:
If the class inherits directly from "object" (which is the case for `class Person`), then there are't really any parents from which to inherit, and this phase is skipped. If the class inherits from one class, then we check there — and on its parents, and its parents, etc.
4. **Object:** All classes in Python inherit, directly or indirectly, from "object", the top of our class hierarchy. As a result, searching for an attribute, if not first found elsewhere always concludes on “object”. If we cannot find an attribute on “object”, and it wasn’t found elsewhere previously, then Python raises an `AttributeError` exception.

(*) Which would mean that there is one unique memory location re-used by all instances.

**Remark:** I can definitely recommand watching the following presentation: https://www.youtube.com/watch?v=Tn1wLsj7Bys

It's important to note that when we run a code like the following, no ICPO lookup is triggered. The only things what happens is that a NEW class attribute is added to the instance.
Consequently, this new instance attribute "shadows" the class attribute when we perform the ICPO lookup.

In [ ]:
p = Person('Mike', 'Tyson')

# This line does not require an ICPO lookup. Instead, we simply add the attribute TITLES to the instance.
p.TITLES = 'asdf'

### Quiz

In [175]:
class Person:
    
    # Titles is now a list!
    TITLES = ['Mr', 'Ms', 'Mrs']

    def __init__(self, first_name, last_name):
        self.first_name = first_name
        self.last_name = last_name
        
        
    def full_name(self):
        return f'{self.first_name} {self.last_name}'
    
    def __str__(self):
        return f'{self.first_name} {self.last_name}' 
    
    def __repr__(self):
        return f'Person(first_name="{self.first_name}", last_name="{self.last_name}")' 
    

In [176]:
p1 = Person('P', '1')

p1.TITLES.extend('-')

p2 = Person('P', '2')

**Question:** What values do the `TITLES` attributes of p1 and p2 have? Explain why.

In [177]:
print(p1.TITLES)
print(p2.TITLES)

['Mr', 'Ms', 'Mrs', '-']
['Mr', 'Ms', 'Mrs', '-']
